In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from typing import Tuple, Dict
from sklearn.linear_model import LassoCV, LassoCV
from scipy.stats import skew, kurtosis

In [8]:
ltcm = pd.read_excel("ltcm_exhibits_data.xlsx",
                     sheet_name="Exhibit 2",
                     header=2,          # row with 'Gross Monthly Performancea', etc.
                     dtype=str)         # keep as strings so we can parse

def parse_pct(col: pd.Series) -> pd.Series:
    """Turn strings like '-1.3%' or ' 1.4' into decimal returns."""
    s = col.str.strip()
    s = s.str.replace(",", "", regex=True)
    s = s.str.replace("%", "", regex=True)
    return pd.to_numeric(s, errors="coerce") / 100.0

gross = parse_pct(ltcm["Gross Monthly Performancea"]).dropna().reset_index(drop=True)
net   = parse_pct(ltcm["Net Monthly Performanceb"]).dropna().reset_index(drop=True)

In [9]:
spy_raw = pd.read_excel("spy_data.xlsx",
                        sheet_name="excess returns",
                        dtype=str)

spy = pd.to_numeric(spy_raw["SPY"], errors="coerce").dropna().reset_index(drop=True)
spy.name = "SPY"

In [10]:
T = min(len(net), len(spy))
net  = net.iloc[:T].reset_index(drop=True)
gross = gross.iloc[:T].reset_index(drop=True)
spy  = spy.iloc[:T].reset_index(drop=True)

In [12]:
#1
def ann_stats(s: pd.Series, freq=12) -> Dict[str, float]:
    v = s.dropna().astype(float)
    alpha = v.mean() * freq
    vol = v.std(ddof=1) * np.sqrt(freq)
    sp = alpha / vol if vol != 0 else np.nan
    sk = skew(v, bias=False)
    ku = kurtosis(v, fisher=False)
    q5 = np.percentile(v, 5)
    return {"Annualized Mean": alpha, "Annualized Volatility": vol, "Sharpe Ratio": sp, "Skewness": sk, "Kurtosis": ku, "5th Percentile (monthly)": q5}

g = ann_stats(gross)
n = ann_stats(net)

pd.DataFrame([g, n], index=["LTCM Gross", "LTCM Net"])

,Annualized Mean,Annualized Volatility,Sharpe Ratio,Skewness,Kurtosis,5th Percentile (monthly)
LTCM Gross,0.002939,0.001364,2.155321,-0.296428,4.314050,-0.000264
LTCM Net,0.002072,0.001119,1.851315,-0.817870,5.527464,-0.000224


In [13]:
#2.
def ann_stats(s: pd.Series, freq=12) -> Dict[str, float]:
    v = s.dropna().astype(float)
    m = v.mean() * freq * 100
    sd = v.std(ddof=1) * np.sqrt(freq) * 100
    sp = m / sd if sd != 0 else np.nan
    sk = skew(v, bias=False)
    ku = kurtosis(v, fisher=False)
    q5 = np.percentile(v, 5)
    return {"Annualized Mean (%)": m, "Annualized Vol (%)": sd, "Sharpe": sp, "Skew": sk, "Kurt": ku, "5th pct": q5}

g_stats = ann_stats(gross)
n_stats = ann_stats(net)
s_stats = ann_stats(spy)

out = pd.DataFrame([g_stats, n_stats, s_stats], index=["LTCM Gross", "LTCM Net", "SPY"])
out["Annualized Mean Diff Gross–Net"] = g_stats["Annualized Mean (%)"] - n_stats["Annualized Mean (%)"]
out

,Annualized Mean (%),Annualized Vol (%),Sharpe,Skew,Kurt,5th pct,Annualized Mean Diff Gross–Net
LTCM Gross,0.293887,0.136354,2.155321,-0.296428,4.314050,-0.000264,0.086717
LTCM Net,0.207170,0.111904,1.851315,-0.817870,5.527464,-0.000224,0.086717
SPY,15.091445,11.504067,1.311836,-0.402156,2.486045,-0.049667,0.086717


The LTCM strategy shows strong positive annualized mean but substantially lower Sharpe than SPY, indicating higher volatility per unit return. Fees reduce mean only moderately from gross to net, while skewness and kurtosis increase, implying that net returns have relatively fatter tails and more downside risk.

In [14]:
#3. 
# align samples safely
T = min(len(net), len(spy))
y = net.iloc[:T].reset_index(drop=True)
mkt = spy.iloc[:T].reset_index(drop=True)
mkt.name = "SPY"

# run OLS
X = sm.add_constant(mkt)
model = sm.OLS(y, X).fit()

alpha = model.params["const"] * 12 * 100
beta = model.params["SPY"] * 100
r2 = model.rsquared

print("annualized alpha (%):", alpha)
print("beta SPY (%):", beta)
print("r-squared:", r2)
print(model.summary())

annualized alpha (%): 0.23324634993376508
beta SPY (%): -0.17279020963310557
r-squared: 0.03155360343667768
                               OLS Regression Results                               
Dep. Variable:     Net Monthly Performanceb   R-squared:                       0.032
Model:                                  OLS   Adj. R-squared:                  0.013
Method:                       Least Squares   F-statistic:                     1.662
Date:                      Sat, 29 Nov 2025   Prob (F-statistic):              0.203
Time:                              23:16:35   Log-Likelihood:                 352.15
No. Observations:                        53   AIC:                            -700.3
Df Residuals:                            51   BIC:                            -696.4
Df Model:                                 1                                         
Covariance Type:                  nonrobust                                         
                 coef    std err          

The regression evidence shows LTCM’s net excess returns have a small positive annualized alpha, meaning it delivered modest performance beyond market risk, but its beta is negative and insignificant and R^2 is low, indicating minimal systematic exposure to SPY and no strong equity market dependence.

In [15]:
#4. 
y = net.copy()
X = pd.DataFrame({"SPY": spy, "SPY2": spy**2})
X = sm.add_constant(X)

d = pd.concat([y, X], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
mod = sm.OLS(d.iloc[:,0], d.iloc[:,1:]).fit()

print("alpha_ann (%):", mod.params["const"]*12*100)
print("beta SPY (%):", mod.params["SPY"]*100)
print("beta SPY^2 (%):", mod.params["SPY2"]*100)
print("r2:", mod.rsquared)
print(mod.summary())

alpha_ann (%): 0.1835642212177706
beta SPY (%): -0.21970656430588048
beta SPY^2 (%): 3.8140212033284855
r2: 0.05373505813376356
                               OLS Regression Results                               
Dep. Variable:     Net Monthly Performanceb   R-squared:                       0.054
Model:                                  OLS   Adj. R-squared:                  0.016
Method:                       Least Squares   F-statistic:                     1.420
Date:                      Sat, 29 Nov 2025   Prob (F-statistic):              0.251
Time:                              23:18:32   Log-Likelihood:                 352.76
No. Observations:                        53   AIC:                            -699.5
Df Residuals:                            50   BIC:                            -693.6
Df Model:                                 2                                         
Covariance Type:                  nonrobust                                         
                 coef 

* Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?

  The quadratic term adds almost no explanatory power, and R^2 increases only marginally, meaning LTCM returns are effectively linear with negligible convexity from SPY^2.

* From the regression evidence, does LTCM's market exposure behave as if it is long market options or short market options?

  The quadratic beta is too small to indicate an options-like convex payoff, so LTCM does not behave like a meaningful long or short convex market options position.

* Should we describe LTCM as being positively or negatively exposed to market volatility?

  With near-zero loading on SPY^2 and low R^2 improvement, LTCM should not be described as materially exposed to equity market volatility under this model.


In [19]:
#6. 
n = min(len(net), len(spy))
y = net.iloc[:n].reset_index(drop=True)
m = spy.iloc[:n].reset_index(drop=True)

k1, k2 = 0.03, -0.03
up = np.maximum(m - k1, 0)
down = np.maximum(k2 - m, 0)

X = pd.DataFrame({"const": 1.0, "market": m, "up": up, "down": down})
model = sm.OLS(y, X).fit()

print(f"Annualized alpha (%): {model.params['const']*12*100:.6f}")
print(f"Market beta: {model.params['market']:.6f}")
print(f"Up beta (βu): {model.params['up']:.6f}")
print(f"Down beta (βd): {model.params['down']:.6f}")
print(f"R²: {model.rsquared:.6f}")
print(model.summary())

Annualized alpha (%): 0.203446
Market beta: -0.002177
Up beta (βu): 0.004162
Down beta (βd): 0.003885
R²: 0.051700
                               OLS Regression Results                               
Dep. Variable:     Net Monthly Performanceb   R-squared:                       0.052
Model:                                  OLS   Adj. R-squared:                 -0.006
Method:                       Least Squares   F-statistic:                    0.8905
Date:                      Sat, 29 Nov 2025   Prob (F-statistic):              0.453
Time:                              23:24:55   Log-Likelihood:                 352.71
No. Observations:                        53   AIC:                            -697.4
Df Residuals:                            49   BIC:                            -689.5
Df Model:                                 3                                         
Covariance Type:                  nonrobust                                         
                 coef    std err   

* Is LTCM long or short the call-like factor? And the put-like factor?

LTCM is effectively short the call-like factor because its up-beta is small and statistically weak, while it is slightly long the put-like factor given the positive but insignificant down-beta.

* Which factor moves LTCM more, the call-like factor or the put-like factor?

The call-like factor (up) has a marginally larger coefficient than the put-like factor (down), but both are small and statistically insignificant, so neither drives large movements.

* Using this current regression, does this volatility exposure come more from being long the market’s upside, short the market’s downside, or something else?

The volatility exposure does not come meaningfully from upside or downside equity convexity and remains mostly unexplained by SPY-based option-like factors.